<a href="https://colab.research.google.com/github/eccho03/datamining_project_clothes/blob/main/%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/MyDrive/데마 프젝"

/content/drive/MyDrive/데마 프젝


In [ ]:
!ls

chromedriver_linux64.zip		  google-chrome-stable_current_amd64.deb.2  크롤링.ipynb
google-chrome-stable_current_amd64.deb	  LICENSE.chromedriver
google-chrome-stable_current_amd64.deb.1  products_info.csv


In [ ]:
!pip install requests

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 26.4 MB/s eta 0:00:00


In [ ]:
!pip install webdriver-manager

In [ ]:
# 기존 Chrome 삭제
!apt-get remove -y google-chrome-stable

# 최신 Google Chrome 설치
!apt-get update
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get install -y -f

# 설치된 Chrome 버전 확인
!google-chrome --version


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package google-chrome-stable
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Get:12 http://s

In [ ]:
!wget -N https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv -f chromedriver /usr/local/bin/chromedriver
!chromedriver --version


--2024-12-02 07:48:49--  https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘chromedriver_linux64.zip’ not modified on server. Omitting download.

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
replace LICENSE.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace LICENSE.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: LICENSE.chromedriver    
ChromeDriver 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


# 데이터 프레임 생성

In [ ]:
import pandas as pd

# 페이지 접근

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
!ls

chromedriver_linux64.zip		  google-chrome-stable_current_amd64.deb.3
google-chrome-stable_current_amd64.deb	  LICENSE.chromedriver
google-chrome-stable_current_amd64.deb.1  products_info.csv
google-chrome-stable_current_amd64.deb.2  크롤링.ipynb


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # GUI 없는 환경에서 사용 (CLI 환경에서 필수)
chrome_options.add_argument('--no-sandbox')  # 권한 문제 방지
chrome_options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 문제 방지
chrome_options.add_argument('--window-size=1920x1080')  # 가상 브라우저 크기 설정

chrome_options.binary_location = '/usr/bin/google-chrome'

# ChromeDriver 서비스 설정
service = Service(ChromeDriverManager().install())

In [ ]:
gender = ""
cate1 = ""
cate2 = ""

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 페이지 스크롤 코드
def scroll_page(driver):
    # 동적할당된 데이터 가져오기 위한 과정(화면 스크롤)
    time.sleep(3)  # 페이지 로드 시간 대기

    # 스크롤 후 데이터가 로드되기까지 대기 시간 설정
    scroll_pause_time = 2  # 페이지가 로드되기 위해 잠시 기다리는 시간

    # 마지막 스크롤 높이 확인
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 최대 시도 횟수 설정 (스크롤의 종료 조건을 위한 안전장치)
    max_attempts = 5
    attempt = 0
    while True:
        # 페이지 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 데이터 로드 대기
        time.sleep(scroll_pause_time)

        # 스크롤 이후의 페이지 높이 측정
        new_height = driver.execute_script("return document.body.scrollHeight")

        # 더 이상 로드할 항목이 없으면 종료
        if new_height == last_height:
            # 추가 시도 횟수 카운트
            attempt += 1
            if attempt >= max_attempts:
                # 최대 시도 횟수에 도달하면 종료
                print("스크롤 완료: 더 이상 새로운 항목이 없습니다.")
                break
            else:
                # 시도 횟수가 남아 있으면 계속 시도
                time.sleep(2)
        else:
            # 새로운 높이로 업데이트하고 시도 횟수 초기화
            last_height = new_height
            attempt = 0  # 시도 횟수 초기화

In [ ]:
import time
def into_url(driver, url):
  retry_count = 0
  while True:
      try:
          driver.get(url)
          break  # 성공하면 루프 종료
      except TimeoutError:
          retry_count += 1
          print(f"타임아웃 발생. 재시도")
          time.sleep(60)  # 1분 대기 후 재시도

In [ ]:
def select_item_tag(url):
  # Chrome 웹 드라이버 실행
  driver = webdriver.Chrome(service=service, options=chrome_options)

  into_url(driver, url)

  scroll_page(driver) # 화면 스크롤

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  '''
  with open('products_info.txt', 'w') as file:
    print(soup.prettify(), file=file)
  '''
  # r : 파싱할 HTML문서
  # html.parser : 파싱에 사용할 파서
  # CSS 선택자 사용
  clothes_detail = soup.select('div.product-card.product-grid__card.css-1tmiin5')

  '''
  with open('products_info2.txt', 'w') as file:
    print(clothes_detail, file=file)
  '''
  return clothes_detail

In [ ]:
def get_product_info(clothes_detail):  # 확인용
  for link in clothes_detail:
    num = link.get('data-product-position')
    link2 = link.find('a', attrs={'class': 'product-card__link-overlay'})
    href = link2.get('href')
    print(num,") url : ", href)

In [ ]:
clothes_list = pd.DataFrame(columns=['page_url', '제품명', '제품 소개', 'cate1', 'cate2', '가격','성별', 'XS이하', 'S', 'M', 'L', 'XL이상'])

In [ ]:
def get_detail_info(url):
  global gender
  global cate
  xs=0
  s=0
  m=0
  l=0
  xl=0
  # Chrome 웹 드라이버 실행
  driver = webdriver.Chrome(service=service, options=chrome_options)

  # url문자열에 {}에 format함수가 받은 파라미터를 차례로 넣어줌
  print("-->", url)
  into_url(driver, url)

  scroll_page(driver) # 화면 스크롤

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  # size
  size_tag = soup.find('div', attrs={'id' : 'size-selector'})
  if size_tag is None: return 0
  size_tag = size_tag.select('div.css-vmnznv.nds-grid-item:not(.disabled)')
  print('tag :',size_tag)
  for tag in size_tag:
    tmp = tag.find('label')
    clothes_size = tmp.text
    clothes_size = clothes_size.strip()
    print('size :', clothes_size)
    if len(clothes_size)>5:
      if 'KR' in clothes_size and 'US' in clothes_size:
        clothes_size = clothes_size.split()
        clothes_size = clothes_size[1]
      else:
        return
    if('XS' in clothes_size):
      xs=1
      print("select XS")
    if(clothes_size == 'S'):
      s=1
      print("select S")
    if(clothes_size == 'M'):
      m=1
      print("select M")
    if(clothes_size == 'L'):
      l=1
      print("select L")
    if('XL' in clothes_size):
      xl=1
      print("select XL")
    #사이즈는 0은 없음 1은 있음
  #제품소개
  title_tag = soup.find('div', attrs={'id' : 'title-container'})
  intro_tag = title_tag.find('h1')
  if intro_tag is None: return
  intro = intro_tag.text
  #제품명
  name_tag = title_tag.find('h2')
  name = name_tag.text
  #가격(원가)
  price_tag = soup.find('div', attrs={'id' : 'price-container'})
  price_tag = price_tag.find_all('span')
  if len(price_tag)!=1 : price = price_tag[1].text
  else : price = price_tag[0].text
  #성별

  #데이터 프레임에 추가
  clothes_list.loc[len(clothes_list)] = [url, name, intro, cate1, cate2, price, gender, xs, s, m, l, xl]
  driver.quit()

In [ ]:
def add_product_info(clothes_detail):
  for link in clothes_detail:
    print(link)
    num = link.get('data-product-position')
    link2 = link.find('a', attrs={'class': 'product-card__link-overlay'})
    print(link2)
    href = link2.get('href')
    get_detail_info(href)

In [ ]:
def select_cate_url(url):
  global cate2
  # Chrome 웹 드라이버 실행
  driver = webdriver.Chrome(service=service, options=chrome_options)

  # 페이지 접근
  print("-->", url)
  into_url(driver, url)

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  # r : 파싱할 HTML문서
  # html.parser : 파싱에 사용할 파서

  cate0 = soup.find('div', attrs={'class' : 'categories__content'})
  cate1 = cate0.select('a.is--link.categories__item.css-g2lskw')
  for tag in cate1:
    cate2 = tag.text
    cate_url = tag.get('href')
    print("cate :",cate2)
    print('url :',cate_url)
    info = select_item_tag(cate_url)
    add_product_info(info)

In [ ]:
gender="W"
cate1="탑 & 티셔츠"
select_cate_url("https://www.nike.com/kr/w/women-tops-tshirts-5e1x6z9om13")

--> https://www.nike.com/kr/w/women-tops-tshirts-5e1x6z9om13
cate : 그래픽 티셔츠
url : https://www.nike.com/kr/w/women-graphic-t-shirts-5e1x6z5shrv
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-칠-니트-여성-슬림-긴팔-크롭-탑-MVpJND1j/HF5323-338" tabindex="-1">나이키 스포츠웨어 칠 니트</a><a aria-label="나이키 스포츠웨어 칠 니트" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-칠-니트-여성-슬림-긴팔-크롭-탑-MVpJND1j/HF5323-338"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/imag

In [ ]:
cate1="후디 & 크루"
select_cate_url("https://www.nike.com/kr/w/women-hoodies-pullovers-5e1x6z6rive")

--> https://www.nike.com/kr/w/women-hoodies-pullovers-5e1x6z6rive
cate : 풀오버
url : https://www.nike.com/kr/w/women-sweatshirts-5e1x6zb43ua
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-클럽-플리스-여성-크루넥-스웻셔츠-CTqPQVRi/HV6499-141" tabindex="-1">나이키 스포츠웨어 클럽 플리스</a><a aria-label="나이키 스포츠웨어 클럽 플리스" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-클럽-플리스-여성-크루넥-스웻셔츠-CTqPQVRi/HV6499-141"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/imag

In [ ]:
cate1="쇼츠"
cate2 = "쇼츠"
info = select_item_tag("https://www.nike.com/kr/w/women-shorts-38fphz5e1x6")
add_product_info(info)

스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/프로-365-여성-8인치-쇼츠-WgfOQ4yP/CZ9841-010" tabindex="-1">나이키 프로 365</a><a aria-label="나이키 프로 365" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/프로-365-여성-8인치-쇼츠-WgfOQ4yP/CZ9841-010"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_592,f_auto/t_product_v1/dc2649d4-3ee8-474b-ae17-4fe8a29d9cdf/AS+W+NP+365+SHORT+8IN.png" width="100%"/><noscript><img alt="undefined undefined" class="prod

In [ ]:
cate1="팬츠 & 타이츠"
select_cate_url("https://www.nike.com/kr/w/women-pants-tights-2kq19z5e1x6")

--> https://www.nike.com/kr/w/women-pants-tights-2kq19z5e1x6
cate : 조거 & 스웻팬츠
url : https://www.nike.com/kr/w/women-joggers-sweatpants-5e1x6zaepf0
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-클럽-플리스-여성-오버사이즈-미드라이즈-스웻팬츠-JxSiC0FU/HV6501-141" tabindex="-1">나이키 스포츠웨어 클럽 플리스</a><a aria-label="나이키 스포츠웨어 클럽 플리스" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-클럽-플리스-여성-오버사이즈-미드라이즈-스웻팬츠-JxSiC0FU/HV6501-141"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https:

In [ ]:
cate1="재킷 & 베스트"
select_cate_url("https://www.nike.com/kr/w/women-jackets-vests-50r7yz5e1x6")

--> https://www.nike.com/kr/w/women-jackets-vests-50r7yz5e1x6
cate : 봄버 재킷
url : https://www.nike.com/kr/w/women-bomber-jackets-1cti3z5e1x6
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-여성-리버서블-인조-퍼-봄버-재킷-EK478hYy/FB8693-133" tabindex="-1">나이키 스포츠웨어</a><a aria-label="나이키 스포츠웨어" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-여성-리버서블-인조-퍼-봄버-재킷-EK478hYy/FB8693-133"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_

In [ ]:
cate1="트랙수트"
select_cate_url("https://www.nike.com/kr/w/women-tracksuits-1ll2wz5e1x6")

--> https://www.nike.com/kr/w/women-tracksuits-1ll2wz5e1x6
cate : 트랙수트 하의
url : https://www.nike.com/kr/w/women-tracksuit-bottoms-5e1x6zadp0n
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/나오미-오사카-여성-하이웨이스트-루즈-니트-팬츠-zZPVpKUX/FV7799-121" tabindex="-1">나오미 오사카</a><a aria-label="나오미 오사카" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/나오미-오사카-여성-하이웨이스트-루즈-니트-팬츠-zZPVpKUX/FV7799-121"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,

In [ ]:
cate2 = "트랙수트 상의"
info = select_item_tag("https://www.nike.com/kr/w/women-tracksuit-tops-5e1x6zad94p")
add_product_info(info)

스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-컬렉션-여성-오버사이즈-리펠-지퍼-재킷-PmgKkNPn/FV7536-014" tabindex="-1">나이키 스포츠웨어 컬렉션</a><a aria-label="나이키 스포츠웨어 컬렉션" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/스포츠웨어-컬렉션-여성-오버사이즈-리펠-지퍼-재킷-PmgKkNPn/FV7536-014"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_592,f_auto/t_product_v1/07e78429-0def-48a4-b7bb-89297dc48a94/AS+W+NSW+CLLCTN+RPL+ZIP+JKT+O.png" width="100%"/><noscript><img a

In [ ]:
cate1="스커트 & 드레스"
cate2 = "스커트 & 드레스"
info = select_item_tag("https://www.nike.com/kr/w/women-skirts-dresses-5e1x6z8y3qp")
add_product_info(info)

스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/acg-로프-드-도프-여성-써마-핏-adv-스커트-z6SJtxGa/HM9500-010" tabindex="-1">나이키 ACG '로프 드 도프'</a><a aria-label="나이키 ACG '로프 드 도프'" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/acg-로프-드-도프-여성-써마-핏-adv-스커트-z6SJtxGa/HM9500-010"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_592,f_auto/t_product_v1/0dd93a6e-e77e-4cc9-a0c9-b8f963684898/AS+W+ACG+TFADV+ROPE+DE+DOPE+SK.png" width="100%"/><noscri

In [ ]:
gender="M"
cate1="탑 & 티셔츠"
select_cate_url("https://www.nike.com/kr/w/men-tops-tshirts-9om13znik1")

--> https://www.nike.com/kr/w/men-tops-tshirts-9om13znik1
cate : 그래픽 티셔츠
url : https://www.nike.com/kr/w/men-graphic-t-shirts-5shrvznik1
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/조던-남성-크루넥-티셔츠-VLdIyvtF/HQ2666-045" tabindex="-1">조던</a><a aria-label="조던" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/조던-남성-크루넥-티셔츠-VLdIyvtF/HQ2666-045"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_592,f_auto/t_product_v1/u_126ab356-44d8

In [ ]:
cate1="후디 & 크루"
select_cate_url("https://www.nike.com/kr/w/men-hoodies-pullovers-6riveznik1")

--> https://www.nike.com/kr/w/men-hoodies-pullovers-6riveznik1
cate : 풀오버
url : https://www.nike.com/kr/w/men-sweatshirts-b43uaznik1
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/클럽-남성-쿼터집-플리스-탑-cukBObS3/FZ0874-010" tabindex="-1">나이키 클럽</a><a aria-label="나이키 클럽" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/클럽-남성-쿼터집-플리스-탑-cukBObS3/FZ0874-010"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_592,f_auto/t_product_v1/59d32e7

In [ ]:
cate1="재킷 & 베스트"
select_cate_url("https://www.nike.com/kr/w/men-jackets-vests-50r7yznik1")

--> https://www.nike.com/kr/w/men-jackets-vests-50r7yznik1
cate : 봄버 재킷
url : https://www.nike.com/kr/w/men-bomber-jackets-1cti3znik1
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/테크-남성-우븐-재킷-99mXvpHF/FZ0752-010" tabindex="-1">나이키 테크</a><a aria-label="나이키 테크" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/테크-남성-우븐-재킷-99mXvpHF/FZ0752-010"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_592,f_auto/t_product_v1/860d9ad6-e53e-

In [ ]:
cate1="팬츠 & 타이츠"
select_cate_url("https://www.nike.com/kr/w/men-pants-tights-2kq19znik1")

--> https://www.nike.com/kr/w/men-pants-tights-2kq19znik1
cate : 조거 & 스웻팬츠
url : https://www.nike.com/kr/w/men-joggers-sweatpants-aepf0znik1
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/조던-플라이트-남성-하이파일-플리스-팬츠-t5q0FviN/FV7450-010" tabindex="-1">조던 플라이트</a><a aria-label="조던 플라이트" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/조던-플라이트-남성-하이파일-플리스-팬츠-t5q0FviN/FV7450-010"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_592,f_a

In [ ]:
cate1="트랙수트"
select_cate_url("https://www.nike.com/kr/w/men-tracksuits-1ll2wznik1")

--> https://www.nike.com/kr/w/men-tracksuits-1ll2wznik1
cate : 트랙수트 상의
url : https://www.nike.com/kr/w/men-tracksuit-tops-ad94pznik1
스크롤 완료: 더 이상 새로운 항목이 없습니다.
<div class="product-card product-grid__card css-1tmiin5" data-product-position="1" data-testid="product-card"><div class="product-card__body" data-el-type="Card" data-testid="product-card__body"><figure><a class="product-card__link-overlay" data-testid="product-card__link-overlay" href="https://www.nike.com/kr/t/테크-윈드러너-남성-우븐-풀집-재킷-qmLY42K7/HM7152-014" tabindex="-1">나이키 테크 윈드러너</a><a aria-label="나이키 테크 윈드러너" class="product-card__img-link-overlay" data-el-type="Hero" data-testid="product-card__img-link-overlay" href="https://www.nike.com/kr/t/테크-윈드러너-남성-우븐-풀집-재킷-qmLY42K7/HM7152-014"><div class="wall-image-loader css-1la3v4n" data-testid="wall-image-loader"><img alt="undefined undefined" class="product-card__hero-image css-1fxh5tw" height="100%" loading="eager" sizes="" src="https://static.nike.com/a/images/c_limit,w_592,f_auto/t_

In [ ]:
cate1="쇼츠"
select_cate_url("https://www.nike.com/kr/w/men-shorts-38fphznik1")

--> https://www.nike.com/kr/w/men-shorts-38fphznik1


In [ ]:
clothes_list.drop(index = clothes_list.index[177:180], inplace=True)

In [ ]:
clothes_list

,page_url,제품명,제품 소개,cate1,cate2,가격,성별,XS이하,S,M,L,XL이상
0,https://www.nike.com/kr/t/조던-남성-크루넥-티셔츠-VLdIyv...,남성 크루넥 티셔츠,조던,탑 & 티셔츠,그래픽 티셔츠,"69,000 원",M,0,1,1,1,1
1,https://www.nike.com/kr/t/테이텀-남성-티셔츠-wduHw5h9/...,남성 티셔츠,테이텀,탑 & 티셔츠,그래픽 티셔츠,"75,000 원",M,0,1,1,1,1
2,https://www.nike.com/kr/t/조던-mvp-남성-긴팔-티셔츠-zD2...,남성 긴팔 티셔츠,조던 MVP,탑 & 티셔츠,그래픽 티셔츠,"65,000 원",M,0,0,0,0,1
3,https://www.nike.com/kr/t/스포츠웨어-남성-오버사이즈-긴팔-티셔...,남성 오버사이즈 긴팔 티셔츠,나이키 스포츠웨어,탑 & 티셔츠,그래픽 티셔츠,"75,000 원",M,0,1,1,1,1
4,https://www.nike.com/kr/t/스포츠웨어-프리미엄-에센셜-남성용-오...,남성용 오버사이즈 티셔츠,나이키 스포츠웨어 프리미엄 에센셜,탑 & 티셔츠,그래픽 티셔츠,"65,000 원",M,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
392,https://www.nike.com/kr/t/클럽-남성-폴리-니트-트랙수트-xLd...,남성 폴리 니트 트랙수트,나이키 클럽,트랙수트,트랙수트 셋업,"109,000 원",M,0,0,0,0,1
393,https://www.nike.com/kr/t/드라이-핏-아카데미-남성-축구-트랙수...,남성 축구 트랙수트,나이키 드라이 핏 아카데미,트랙수트,트랙수트 셋업,"95,000 원",M,0,0,1,1,1
394,https://www.nike.com/kr/t/조던-스포츠-잼-남성-웜업-팬츠-VF...,남성 웜업 팬츠,조던 스포츠 잼,트랙수트,트랙수트 셋업,"119,000 원",M,1,1,1,1,1
395,https://www.nike.com/kr/t/조던-스포츠-잼-남성-워밍업-재킷-D...,남성 워밍업 재킷,조던 스포츠 잼,트랙수트,트랙수트 셋업,"145,000 원",M,0,1,1,1,1


In [ ]:
clothes_list.to_csv('products_info_women.csv', index=False, encoding='utf-8')

In [ ]:
clothes_list.to_csv('products_info_men.csv', index=False, encoding='utf-8')